In [ ]:
!pip install keras-tuner

import tensorflow as tf
from tensorflow import keras
import numpy as np

fashion_mnist=keras.datasets.fashion_mnist

(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

train_images=train_images/255.0
test_images=test_images/255.0

train_images[0].shape

train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv-2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
      
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model
                


from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

model=tuner_search.get_best_models(num_models=1)[0]

model.summary()


INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |128               |?                 
conv_1_kernel     |5                 |?                 
conv-2_filter     |64                |?                 
conv_2_kernel     |5                 |?                 
dense_1_units     |64                |?                 
learning_rate     |0.01              |?                 

Epoch 1/3
1688/1688 [==============================] - 741s 439ms/step - loss: 0.6086 - accuracy: 0.8121 - val_loss: 0.4038 - val_accuracy: 0.8498
Epoch 2/3
1688/1688 [==============================] - 745s 441ms/step - loss: 0.3997 - accuracy: 0.8525 - val_loss: 0.4221 - val_accuracy: 0.8438
Epoch 3/3
 218/1688 [==>...........................] - ETA: 10:30 - loss: 0.3670 - accuracy: 0.8602